In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split


In [ ]:
from google.colab import files

uploaded = files.upload()

Saving strong_lense_train.csv to strong_lense_train.csv


In [ ]:
# Load data
df = pd.read_csv('strong_lense_train.csv')
print(len(df))

3329


In [ ]:
# catalog = pd.read_csv('q1_discovery_engine_lens_catalog(1).csv')
catalog = df
catalog['expert_score'] = catalog['label']

In [ ]:
catalog['id_str']


,id_str
0,102018212_-588199575513249872
1,102018213_-594971757513738267
2,102018664_-563268036507955941
3,102018664_-566148966507539414
4,102018664_-569003084507675615
...,...
3324,102160612_2745259655683405723
3325,102160612_2745634053683302741
3326,102160612_2745727028683190439
3327,102160612_2746348900682544986


In [ ]:

catalog['id_str'] = catalog['id_str'].str.replace('NEG', '-')
df = pd.merge(df, catalog[['id_str', 'expert_score']], how='inner')
len(df)

3329

In [ ]:
df['id_str']

,id_str
0,102018212_-588199575513249872
1,102018213_-594971757513738267
2,102018664_-563268036507955941
3,102018664_-566148966507539414
4,102018664_-569003084507675615
...,...
3324,102160612_2745259655683405723
3325,102160612_2745634053683302741
3326,102160612_2745727028683190439
3327,102160612_2746348900682544986


In [ ]:
# (df['expert_score'] > 1.5).sum()
# df['label'].value_counts()

# I think you used > 1.5 as a lens? numbers match (almost)

label_bin_cutoff = 0.5
df['label'] = df['expert_score'] > label_bin_cutoff

In [ ]:

# just to help with class imbalance, drop the bottom 50%?

# import numpy as np
# expert_score_cut = np.percentile(df['expert_score'], 50)

# df = df[df['expert_score'] > expert_score_cut].reset_index(drop=True)

In [ ]:

X = df[[col for col in df.columns.values if col.startswith('feat')]].values
# y = df['expert_score'] / df['expert_score'].max()
y = df['expert_score'].values


# Separate features X (first 40 columns) and target y (the 41st column)
# X = df.iloc[:, 1:41].values   # shape: (N, 40)
# y = df.iloc[:, 41].values    # shape: (N,)


In [ ]:
# y.min(), y.max()
# import matplotlib.pyplot as plt
# _ = plt.hist(y)



In [ ]:

# Train-test split (no stratify to avoid errors with extremely small classes)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=38
)

In [ ]:
# and the classification equivalent
y_train_class = y_train > label_bin_cutoff
y_test_class = y_test > label_bin_cutoff

pd.value_counts(y_train_class)
# pd.value_counts(y_test_class)

<ipython-input-42-fea0e6d8b5a8>:5: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(y_train_class)


,count
False,2143
True,187


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Define simple MLP model (1 hidden layer, 128 units)
mlp = MLPClassifier(hidden_layer_sizes=(128,), activation='relu', solver='adam', max_iter=100, random_state=42)

# Train the model
mlp.fit(X_train, y_train_class)

# Predict probabilities for the test set
y_pred_probs = mlp.predict_proba(X_test)[:, 1]  # Probabilities for class 1

# Get the top 100 samples with the highest probabilities
top_100_indices = np.argsort(-y_pred_probs)[:100]  # Sort in descending order
top_100_labels = y_test_class[top_100_indices]  # Get actual class labels of top 100

# Count how many of the top 100 are actually class 1
num_class_1 = np.sum(top_100_labels)

# Print results
print(f"Out of the top 100 highest probability samples, {num_class_1} are actually class 1.")


Out of the top 100 highest probability samples, 38 are actually class 1.


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


#**Baseline model, cross entropy nn**

#**Focal loss**


$$
FL(p_t) = -\alpha_t (1-p_t)^\gamma \log(p_t)
$$

**here $-α$ is -0.75 when y = 1 and -0.25 when y = 0.
$γ$ is 2.**


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_class, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_class, dtype=torch.float32)

# Define a simple MLP
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=128):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, 1)  # Output a single score (logit)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x  # Raw logits (we'll apply sigmoid in loss function)

# Implement Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.bce = nn.BCEWithLogitsLoss(reduction='none')  # We compute loss manually

    def forward(self, logits, targets):
        """ Compute Focal Loss for binary classification """
        targets = targets.view(-1)  # 🔥 Ensure targets have shape (batch_size,)
        probs = torch.sigmoid(logits)  # Convert logits to probabilities

        # Compute standard BCE loss
        bce_loss = self.bce(logits, targets)

        # Compute focal loss term
        pt = probs * targets + (1 - probs) * (1 - targets)  # P_t = P if y=1, 1-P if y=0
        focal_weight = (1 - pt) ** self.gamma

        # Apply alpha-balancing factor
        alpha_factor = self.alpha * targets + (1 - self.alpha) * (1 - targets)

        # Final loss
        focal_loss = alpha_factor * focal_weight * bce_loss
        return focal_loss.mean()

# Initialize model, optimizer, and loss function
model = MLP(input_dim=40, hidden_dim=64)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = FocalLoss(alpha=0.25, gamma=2.0)
# Training loop
epochs = 500
batch_size = 128
for epoch in range(epochs):
    model.train()
    total_loss = 0

    # Mini-batch training
    for i in range(0, len(X_train_tensor), batch_size):
        batch_X = X_train_tensor[i:i+batch_size]
        batch_y = y_train_tensor[i:i+batch_size]

        optimizer.zero_grad()
        logits = model(batch_X).squeeze()  # Get raw scores (logits)
        loss = criterion(logits, batch_y.view(-1))  # 🔥 Fix shape issue
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}, Avg Train Loss: {total_loss / (len(X_train_tensor)):.4f}")

# Evaluation: Get probabilities and rank top 100
model.eval()
with torch.no_grad():
    logits_test = model(X_test_tensor).squeeze()
    probs_test = torch.sigmoid(logits_test).numpy()  # Convert logits to probabilities

test_loss = criterion(logits_test, y_test_tensor)
print(f'Test Loss: {total_loss / (len(logits_test) // batch_size):.4f}')

# Get the top 100 samples with the highest probabilities
top_100_indices = np.argsort(-probs_test)[:100]  # Sort in descending order
top_100_labels = y_test_class[top_100_indices]  # Get actual class labels of top 100

# Count how many of the top 100 are actually class 1
num_class_1 = np.sum(top_100_labels)

# Print results
print(f"Out of the top 100 highest probability samples, {num_class_1} are actually class 1.")


Epoch 50, Avg Train Loss: 0.0001
Epoch 100, Avg Train Loss: 0.0000
Epoch 150, Avg Train Loss: 0.0000
Epoch 200, Avg Train Loss: 0.0000
Epoch 250, Avg Train Loss: 0.0000
Epoch 300, Avg Train Loss: 0.0000
Epoch 350, Avg Train Loss: 0.0000
Epoch 400, Avg Train Loss: 0.0000
Epoch 450, Avg Train Loss: 0.0000
Epoch 500, Avg Train Loss: 0.0000
Test Loss: 0.0000
Out of the top 100 highest probability samples, 42.0 are actually class 1.


# **Margin Ranking Loss**

Mathematically, Margin Ranking Loss is defined on **pairs of examples** with a label indicating their desired order. For a single pair of items with model output scores $x_1$ and $x_2$, and a target label $y \in \{1, -1\}$, the **per-pair** loss is given by:

$$
L(x_1, x_2, y) = \max (0, -y \cdot (x_1 - x_2) + m).
$$

### **Explanation of the Formula**
- **$x_1, x_2$**: The scores predicted by the model for the two items being compared. A higher score means the item is ranked more highly (e.g., more relevant or more preferred).

- **$y$**: The **label** indicating the correct ranking for the pair. Conventionally:
  - $y = 1$ if **item 1 should be ranked higher** than item 2.
  - $y = -1$ if **item 2 should be ranked higher** than item 1.

  In other words, $y$ tells us which item is supposed to "win" in the comparison.

- **$m$**: The **margin** (a non-negative hyperparameter) that defines the required difference between scores.
  - If the higher-ranked item’s score is at least **$m$** greater than the lower-ranked item’s score, the loss for that pair will be **zero**.
  - A margin of $m = 0$ means we only care about getting the order correct.
  - A positive margin **enforces a gap** between the scores, ensuring the ranking is not only correct but also has sufficient separation.


In [ ]:
class PairwiseRankingDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        """
        X: NumPy array of shape (N, D)
        y: NumPy array of shape (N,)
        num_pairs: number of pairs to sample
        """
        super().__init__()

        self.num_samples = len(X)

        self.X = torch.tensor(X, dtype=torch.float)
        self.y = torch.tensor(y, dtype=torch.float)


    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # Randomly get two samples
        indices = np.random.choice(range(self.num_samples), size=2, replace=False)
        i = indices[0]
        j = indices[1]

        x_i = self.X[i]  # shape: (40,)
        x_j = self.X[j]  # shape: (40,)

        y_i = self.y[i]
        y_j = self.y[j]

        # Our margin-ranking 'label' is always +1 => x_i should rank higher than x_j.
        target = 1 if y_i > y_j else 0



        return x_i, x_j, target


In [ ]:
train_dataset = PairwiseRankingDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = PairwiseRankingDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)



In [ ]:
next(iter(train_loader))

[tensor([[-7.7379e+00, -7.1560e-01,  1.0268e+00,  ...,  6.4300e-01,
          -4.8105e-01,  2.5116e-01],
         [-2.5668e+00,  5.4763e+00,  3.9950e+00,  ..., -1.3153e+00,
          -1.7775e+00,  8.5259e-01],
         [-1.6074e+00,  4.8418e+00, -5.2429e+00,  ..., -6.0304e-01,
          -1.3523e-03, -8.5678e-02],
         ...,
         [-1.3567e+00, -2.7922e+00,  4.3208e+00,  ...,  7.4573e-01,
          -1.6440e-01, -6.0536e-01],
         [-6.7275e+00,  4.5620e+00, -1.0955e+00,  ..., -9.0227e-02,
           4.2628e-01,  2.0157e-01],
         [-1.0888e+00, -1.8525e+00, -1.5739e-01,  ..., -1.8716e-01,
           8.4380e-01,  5.0495e-01]]),
 tensor([[-8.3106,  0.5640,  0.2281,  ...,  0.8582, -0.1769, -0.1098],
         [ 3.5834,  8.9819,  8.5525,  ..., -0.8339,  0.1853,  0.6552],
         [-3.4402,  5.6220,  2.1022,  ...,  0.1987,  0.4750, -0.0384],
         ...,
         [ 2.7290, -6.0325,  4.2567,  ...,  0.3401,  1.0115,  0.0376],
         [-5.1121,  2.4035, -4.2935,  ..., -0.3206, -0.1

In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self, input_dim=40, hidden_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)   # single output -> "score"
        )

    def forward(self, x):
        return self.net(x)




In [ ]:
y_train.min(), y_train.max()

(np.float64(0.0), np.float64(1.0))

In [ ]:

# Instantiate the model
model = SimpleMLP(input_dim=40, hidden_dim=128)

criterion = nn.MarginRankingLoss(margin=0.)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 250

model.train()
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for batch in train_loader:
        x_i_batch, x_j_batch, target_batch = batch
        # x_i_batch: (batch_size, 40)
        # x_j_batch: (batch_size, 40)
        # target_batch: should be (batch_size,) after we fix below.

        # 1. Forward pass
        score_i = model(x_i_batch).squeeze()  # shape: (batch_size,)
        score_j = model(x_j_batch).squeeze()  # shape: (batch_size,)

        # 2. Fix the shape of target_batch
        # If target_batch is (batch_size, 1), we can squeeze it:
        target_batch = target_batch.squeeze()  # shape: (batch_size,)

        # 3. Compute loss
        loss = criterion(score_i, score_j, target_batch)

        # 4. Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

Epoch 50, Loss: 0.1020
Epoch 100, Loss: 0.0265
Epoch 150, Loss: 0.0033
Epoch 200, Loss: 0.0005
Epoch 250, Loss: 0.0028


In [ ]:
model.eval()

# Convert X_test to torch
X_test_tensor = torch.tensor(X_test, dtype=torch.float)

test_loss = 0.
for batch in test_loader:
    x_i_batch, x_j_batch, target_batch = batch
    score_i = model(x_i_batch).squeeze()  # shape: (batch_size,)
    score_j = model(x_j_batch).squeeze()  # shape: (batch_size,)
    target_batch = target_batch.squeeze()  # shape: (batch_size,)
    loss = criterion(score_i, score_j, target_batch)
    test_loss += loss.item()


with torch.no_grad():
    scores_test = model(X_test_tensor).squeeze().numpy()  # shape: (N_test,)

# Sort test samples by descending score
sorted_indices = np.argsort(scores_test)[::-1]

top_100_indices = sorted_indices[:100]

X_test_top_100 = X_test[top_100_indices]
y_test_top_100 = y_test[top_100_indices]
scores_top_100 = scores_test[top_100_indices]

# Count how many of these top 100 are actually class 1
num_correct_in_top_100 = np.sum(y_test_top_100 > label_bin_cutoff)

print("Number of samples in top-100 that are actual class=1:", num_correct_in_top_100)
print("Fraction correct among top-100:", num_correct_in_top_100 / 100)
print(test_loss)

Number of samples in top-100 that are actual class=1: 6
Fraction correct among top-100: 0.06
0.0037090007681399584


In [ ]:
y_test_top_100

array([0.03703704, 0.        , 0.        , 0.28057653, 0.03333334,
       0.13333334, 0.06666667, 0.        , 0.        , 0.15815203,
       0.38345939, 0.08333334, 0.04166667, 0.23378076, 0.21618381,
       0.3168126 , 0.        , 0.24735722, 0.        , 0.38841432,
       0.23521318, 0.03333334, 0.28436854, 0.06666667, 0.3091011 ,
       0.        , 0.59802109, 0.17782824, 0.19005062, 0.25534552,
       0.38067093, 0.13333334, 0.03333334, 0.        , 0.1235102 ,
       0.        , 0.0952381 , 0.32254475, 0.32614326, 0.27308294,
       0.1       , 0.23671941, 0.03333334, 0.        , 0.45220011,
       0.13333334, 0.1968489 , 0.24341507, 0.15600733, 0.13333334,
       0.07407407, 0.24089758, 0.03333334, 0.13333334, 0.32955346,
       0.1       , 0.11111111, 0.19307889, 0.46407142, 0.        ,
       0.15593928, 0.33474764, 0.        , 0.06666667, 0.29677227,
       0.35607985, 0.        , 0.54397428, 0.15151516, 0.06666667,
       0.        , 0.23481071, 0.        , 0.13333334, 0.72546

# **Rank Net Ranking Loss, with PyTorch**

Given a set of training examples $ \{(x_1, y_1), (x_2, y_2), ..., (x_n, y_n)\} $, we consider all possible **ordered pairs** $ (i, j) $ where $ i \neq j $, and we compute the **pairwise loss**:

$$
L = \sum_{i < j} - y_{ij} \log P(i > j) - (1 - y_{ij}) \log (1 - P(i > j))
$$

where:

- $ y_{ij} = 1 $ if $ x_i $ should be ranked higher than $ x_j $, otherwise $ y_{ij} = 0 $.
- **In here since all true labels has prob 1, we only consider the loss between true and false ones**.
- $ P(i > j) = \frac{1}{1 + e^{-(s(x_i) - s(x_j))}} $ is the probability that $ x_i $ is ranked higher than $ x_j $. **s is the rankning score assigned to i, higher s higher ranking.**
- The sum runs over all possible pairs $ (i, j) $ with $ i < j $.




In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random
from itertools import combinations

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoader for batching
batch_size = 256
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Define a simple neural network for ranking (1 hidden layer, 128 units)
class RankNet(nn.Module):
    def __init__(self, input_dim, hidden_dim=128):
        super(RankNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, 1)  # Output raw ranking scores

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x  # Output raw scores (no Sigmoid, because we use ranking loss)

# Use PyTorch's built-in margin ranking loss
def ranknet_loss(y_pred, y_true):
    """
    Efficient pairwise RankNet loss using margin ranking loss.
    - y_pred: Predicted scores
    - y_true: Ground truth binary labels
    """
    indices = torch.arange(len(y_true))  # Index all elements in the batch
    sampled_pairs = list(combinations(indices.tolist(), 2))  # Sample pairs

    if not sampled_pairs:
        return torch.tensor(0.0, requires_grad=True)

    i, j = zip(*sampled_pairs)  # Unpack sampled pairs
    i, j = torch.tensor(i), torch.tensor(j)

    y_i, y_j = y_true[i], y_true[j]
    pred_i, pred_j = y_pred[i], y_pred[j]

    # valid_pairs = (y_i != y_j)  # Only keep pairs with different labels
    # if valid_pairs.sum() == 0:
    #     return torch.tensor(0.0, requires_grad=True)

    y_target = torch.where(y_i > y_j, torch.ones_like(y_i), -torch.ones_like(y_i))  # Set target for ranking loss

    return nn.MarginRankingLoss(margin=1.0)(pred_i, pred_j, y_target)  # Optimized ranking loss

# Initialize model, optimizer
model = RankNet(input_dim=40, hidden_dim=128)
optimizer = optim.Adam(model.parameters(), lr=0.005)

# Training loop
epochs = 500
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()

        y_pred = model(X_batch).squeeze()  # Get predicted scores
        loss = ranknet_loss(y_pred, y_batch)  # Compute ranking loss

        if loss.requires_grad:
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}, Avg Loss: {total_loss / len(train_loader):.4f}")

# Evaluate model and rank samples
model.eval()
y_pred_scores = model(X_test_tensor).detach().numpy().flatten()  # Get predicted scores
ranking = np.argsort(-y_pred_scores)  # Rank in descending order

# Create ranked DataFrame
ranked_df = pd.DataFrame({'Sample': np.arange(len(y_test)), 'True Class': y_test, 'Predicted Score': y_pred_scores})
ranked_df = ranked_df.iloc[ranking].reset_index(drop=True)
print(ranked_df.head(100))  # Show top-ranked samples

Epoch 50, Avg Loss: 0.3647
Epoch 100, Avg Loss: 0.2995
Epoch 150, Avg Loss: 0.2679
Epoch 200, Avg Loss: 0.2537
Epoch 250, Avg Loss: 0.2490
Epoch 300, Avg Loss: 0.2415
Epoch 350, Avg Loss: 0.2376
Epoch 400, Avg Loss: 0.2458
Epoch 450, Avg Loss: 0.2220
Epoch 500, Avg Loss: 0.2551
    Sample  True Class  Predicted Score
0      596    0.833914        13.416726
1      997    0.352183        10.396317
2      197    0.000000        10.390798
3      624    0.372656        10.267813
4      141    0.608551         9.348565
..     ...         ...              ...
95     650    0.887296         5.348153
96     662    0.433943         5.329562
97     855    0.233781         5.326482
98     929    0.292134         5.321490
99     106    0.167950         5.302687

[100 rows x 3 columns]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoader for batching
batch_size = 128
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Define a simple neural network for ranking (1 hidden layer, 128 units)
class RankNet(nn.Module):
    def __init__(self, input_dim, hidden_dim=128):
        super(RankNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, 1)  # Output raw ranking scores

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x  # Output raw scores (no Sigmoid, because we use ranking loss)

# Use PyTorch's built-in margin ranking loss
def ranknet_loss(y_pred, y_true):
    """
    Efficient pairwise RankNet loss using margin ranking loss.
    - y_pred: Predicted scores
    - y_true: Ground truth binary labels
    """
    n = len(y_true)
    indices = torch.arange(n)  # Index all elements in the batch

    # Generate valid pairs (y_i != y_j)
    sampled_pairs = [(i, j) for i, j in combinations(indices.tolist(), 2) if y_true[i] != y_true[j]]

    if len(sampled_pairs) == 0:
        return torch.tensor(0.0, requires_grad=True)

    i, j = zip(*sampled_pairs)  # Unpack sampled pairs
    i, j = torch.tensor(i), torch.tensor(j)

    y_i, y_j = y_true[i], y_true[j]
    pred_i, pred_j = y_pred[i], y_pred[j]

    # Target labels for ranking loss: +1 if y_i > y_j, else -1
    y_target = torch.where(y_i > y_j, torch.ones_like(y_i), -torch.ones_like(y_i))

    # Use MarginRankingLoss
    return nn.MarginRankingLoss(margin=1.0)(pred_i, pred_j, y_target)  # Optimized ranking loss

# Initialize model, optimizer
model = RankNet(input_dim=40, hidden_dim=128)
optimizer = optim.Adam(model.parameters(), lr=0.005)

# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()

        y_pred = model(X_batch).squeeze()  # Get predicted scores
        loss = ranknet_loss(y_pred, y_batch)  # Compute ranking loss

        if loss.requires_grad:
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}, Avg Loss: {total_loss / len(train_loader):.4f}")

# Evaluate model and rank samples
model.eval()
y_pred_scores = model(X_test_tensor).detach().numpy().flatten()  # Get predicted scores
ranking = np.argsort(-y_pred_scores)  # Rank in descending order

# Create ranked DataFrame
ranked_df = pd.DataFrame({'Sample': np.arange(len(y_test)), 'True Class': y_test, 'Predicted Score': y_pred_scores})
ranked_df = ranked_df.iloc[ranking].reset_index(drop=True)
print(ranked_df.head(100))  # Show top-ranked samples


Epoch 50, Avg Loss: 0.1724
    Sample  True Class  Predicted Score
0      596    0.833914         5.659933
1      370    1.000000         4.134814
2      151    0.226369         4.130959
3      110    0.740053         4.049060
4       15    1.000000         3.839596
..     ...         ...              ...
95      21    0.100000         1.661579
96     699    0.216813         1.652558
97     865    0.553375         1.636178
98     210    0.369869         1.634777
99     176    0.696001         1.625689

[100 rows x 3 columns]


In [ ]:
print((ranked_df.head(100)["True Class"] >= 0.5).sum())
print(y_test.sum())

37
150.91162309609354


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from itertools import combinations
import torch.nn.functional as F

# Define simple neural network for ranking (RankNet)
class RankNet(nn.Module):
    def __init__(self, input_dim, hidden_dim=128):
        super(RankNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, 1)  # Output raw ranking scores

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x  # Output raw scores (no Sigmoid, because we use ranking loss)

# RankNet loss
# def ranknet_loss(y_pred, y_true):
#     n = len(y_true)
#     indices = torch.arange(n)  # Index all elements in the batch

#     # Generate valid pairs (y_i != y_j)
#     sampled_pairs = [(i, j) for i, j in combinations(indices.tolist(), 2)]

#     if len(sampled_pairs) == 0:
#         return torch.tensor(0.0, requires_grad=True)

#     i, j = zip(*sampled_pairs)  # Unpack sampled pairs
#     i, j = torch.tensor(i), torch.tensor(j)

#     y_i, y_j = y_true[i], y_true[j]
#     pred_i, pred_j = y_pred[i], y_pred[j]

#     # Target labels for ranking loss: +1 if y_i > y_j, else -1
#     y_target = torch.where(y_i > y_j, torch.ones_like(y_i), -torch.ones_like(y_i))

#     # Use MarginRankingLoss
#     return nn.MarginRankingLoss(margin=1.0)(pred_i, pred_j, y_target)  # Optimized ranking loss

def ranknet_loss(y_pred, y_true):
    n = len(y_true)
    indices = torch.arange(n)

    # Generate valid pairs where y_true[i] != y_true[j]
    sampled_pairs = [(i, j) for i, j in combinations(indices.tolist(), 2) if y_true[i] != y_true[j]]

    if len(sampled_pairs) == 0:
        return torch.tensor(0.0, requires_grad=True)

    # Unpack the sampled pairs
    i, j = zip(*sampled_pairs)
    i = torch.tensor(i)
    j = torch.tensor(j)

    # Get predicted scores and true labels for the pairs
    s_i, s_j = y_pred[i], y_pred[j]
    y_i, y_j = y_true[i], y_true[j]

    # Binary target: 1 if y_i > y_j, else 0
    target = (y_i > y_j).float()

    # Score difference
    diff = s_i - s_j

    # Compute RankNet loss using binary cross-entropy with logits
    loss = F.binary_cross_entropy_with_logits(diff, target)

    return loss

# Set random seed for reproducibility
random_seed = 42

# Number of splits (10)
num_splits = 10
mlp_correct_predictions = []
ranknet_correct_predictions = []

for split in range(num_splits):
    print(f"Split {split+1}/{num_splits}")

    # Split the data with different random states
    random_state = random_seed + split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

    # Train and evaluate MLP model
    mlp = MLPClassifier(hidden_layer_sizes=(128,), activation='relu', solver='adam', max_iter=800, random_state=random_state)
    y_train_class = y_train > label_bin_cutoff
    mlp.fit(X_train, y_train_class)

    # Predict probabilities for the test set
    y_pred_probs = mlp.predict_proba(X_test)[:, 1]  # Probabilities for class 1

    # Get the top 100 samples with the highest probabilities
    top_100_indices = np.argsort(-y_pred_probs)[:100]  # Sort in descending order
    top_100_labels = y_test[top_100_indices]  # Get actual class labels of top 100

    # Count how many of the top 100 are actually class 1
    num_class_1_mlp = np.sum(top_100_labels > 0.5)
    mlp_correct_predictions.append(num_class_1_mlp)

    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

    # Create DataLoader for batching
    batch_size = 128
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Initialize RankNet model
    model = RankNet(input_dim=X_train.shape[1], hidden_dim=128)
    optimizer = optim.Adam(model.parameters(), lr=0.005)

    # Training loop
    epochs = 80
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()

            y_pred = model(X_batch).squeeze()  # Get predicted scores
            loss = ranknet_loss(y_pred, y_batch)  # Compute ranking loss

            if loss.requires_grad:
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

    # Evaluate RankNet model and rank samples
    model.eval()
    y_pred_scores = model(X_test_tensor).detach().numpy().flatten()  # Get predicted scores
    ranking = np.argsort(-y_pred_scores)  # Rank in descending order

    # Get the top 100 ranked samples
    ranked_top_100_labels = y_test[ranking[:100]]  # Get actual class labels of top 100
    num_class_1_ranknet = np.sum(ranked_top_100_labels > 0.5)
    ranknet_correct_predictions.append(num_class_1_ranknet)

# Report results
print("\nPerformance across 10 splits:")

# MLP Model: number of class 1 predictions in top 100 for each split
print(f"MLP model - Correct predictions in top 100:")
for i, correct in enumerate(mlp_correct_predictions, 1):
    print(f"Split {i}: {correct} correct")

# RankNet Model: number of class 1 predictions in top 100 for each split
print(f"\nRankNet model - Correct predictions in top 100:")
for i, correct in enumerate(ranknet_correct_predictions, 1):
    print(f"Split {i}: {correct} correct")

# Average performance across all splits
print(f"\nMLP Average Correct Predictions in Top 100: {np.mean(mlp_correct_predictions)}")
print(f"RankNet Average Correct Predictions in Top 100: {np.mean(ranknet_correct_predictions)}")


Split 1/10
Split 2/10
Split 3/10


KeyboardInterrupt: 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from itertools import combinations
import torch.nn.functional as F

# Define RankNet model and its loss function
class RankNet(nn.Module):
    def __init__(self, input_dim, hidden_dim=128):
        super(RankNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, 1)  # Output raw ranking scores

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x  # Raw scores

def ranknet_loss(y_pred, y_true):
    n = len(y_true)
    indices = torch.arange(n)
    # Generate valid pairs where labels differ
    sampled_pairs = [(i, j) for i, j in combinations(indices.tolist(), 2) if y_true[i] != y_true[j]]
    if len(sampled_pairs) == 0:
        return torch.tensor(0.0, requires_grad=True)
    i, j = zip(*sampled_pairs)
    i = torch.tensor(i)
    j = torch.tensor(j)
    s_i, s_j = y_pred[i], y_pred[j]
    y_i, y_j = y_true[i], y_true[j]
    target = (y_i > y_j).float()  # Binary target: 1 if y_i > y_j else 0
    diff = s_i - s_j
    loss = F.binary_cross_entropy_with_logits(diff, target)
    return loss

# Define a simple MLP for focal loss training
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=64):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, 1)  # Output a single logit

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x  # Raw logits

# Implement Focal Loss for binary classification
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.bce = nn.BCEWithLogitsLoss(reduction='none')

    def forward(self, logits, targets):
        targets = targets.view(-1)  # Ensure shape (batch_size,)
        probs = torch.sigmoid(logits)
        bce_loss = self.bce(logits, targets)
        pt = probs * targets + (1 - probs) * (1 - targets)  # p_t
        focal_weight = (1 - pt) ** self.gamma
        alpha_factor = self.alpha * targets + (1 - self.alpha) * (1 - targets)
        focal_loss = alpha_factor * focal_weight * bce_loss
        return focal_loss.mean()

# Set random seed for reproducibility
random_seed = 42

# Number of splits (10)
num_splits = 10
mlp_correct_predictions = []
ranknet_correct_predictions = []
focal_loss_correct_predictions = []

for split in range(num_splits):
    print(f"\n===== Split {split+1}/{num_splits} =====")

    # Split the data with different random states
    random_state = random_seed + split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

    # Create binary labels for classification based on your cutoff
    y_train_class = (y_train > label_bin_cutoff).astype(np.float32)
    y_test_class = (y_test > label_bin_cutoff).astype(np.float32)

    # ----- Sklearn MLP Model -----
    mlp = MLPClassifier(hidden_layer_sizes=(128,), activation='relu', solver='adam', max_iter=800, random_state=random_state)
    mlp.fit(X_train, y_train_class)
    y_pred_probs = mlp.predict_proba(X_test)[:, 1]  # Probabilities for class 1
    top_100_indices = np.argsort(-y_pred_probs)[:100]
    top_100_labels = y_test[top_100_indices]
    num_class_1_mlp = np.sum(top_100_labels > 0.5)
    mlp_correct_predictions.append(num_class_1_mlp)

    # ----- RankNet Model -----
    # Convert training and test data to torch tensors for RankNet
    X_train_tensor_rank = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor_rank = torch.tensor(y_train, dtype=torch.float32)
    X_test_tensor_rank = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor_rank = torch.tensor(y_test, dtype=torch.float32)

    batch_size = 128
    train_dataset_rank = TensorDataset(X_train_tensor_rank, y_train_tensor_rank)
    train_loader_rank = DataLoader(train_dataset_rank, batch_size=batch_size, shuffle=True)

    model_rank = RankNet(input_dim=X_train.shape[1], hidden_dim=128)
    optimizer_rank = optim.Adam(model_rank.parameters(), lr=0.005)
    epochs_rank = 100
    for epoch in range(epochs_rank):
        model_rank.train()
        total_loss_rank = 0
        for X_batch, y_batch in train_loader_rank:
            optimizer_rank.zero_grad()
            y_pred = model_rank(X_batch).squeeze()
            loss = ranknet_loss(y_pred, y_batch)
            if loss.requires_grad:
                loss.backward()
                optimizer_rank.step()
                total_loss_rank += loss.item()
    model_rank.eval()
    y_pred_scores = model_rank(X_test_tensor_rank).detach().numpy().flatten()
    ranking = np.argsort(-y_pred_scores)
    ranked_top_100_labels = y_test[ranking[:100]]
    num_class_1_ranknet = np.sum(ranked_top_100_labels > 0.5)
    ranknet_correct_predictions.append(num_class_1_ranknet)

    # ----- Focal Loss Model -----
    # Convert training and test data to torch tensors for focal loss model
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor_class = torch.tensor(y_train_class, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor_class = torch.tensor(y_test_class, dtype=torch.float32)

    model_focal = MLP(input_dim=X_train.shape[1], hidden_dim=64)
    optimizer_focal = optim.Adam(model_focal.parameters(), lr=0.001)
    criterion_focal = FocalLoss(alpha=0.25, gamma=2.0)

    epochs_focal = 100
    focal_batch_size = 128
    train_dataset_focal = TensorDataset(X_train_tensor, y_train_tensor_class)
    train_loader_focal = DataLoader(train_dataset_focal, batch_size=focal_batch_size, shuffle=True)

    for epoch in range(epochs_focal):
        model_focal.train()
        total_loss_focal = 0
        for X_batch, y_batch in train_loader_focal:
            optimizer_focal.zero_grad()
            logits = model_focal(X_batch).squeeze()
            loss = criterion_focal(logits, y_batch.view(-1))
            loss.backward()
            optimizer_focal.step()
            total_loss_focal += loss.item()
        if (epoch + 1) % 50 == 0:
            avg_loss = total_loss_focal / len(train_loader_focal)
            print(f"Focal Loss Model - Epoch {epoch+1}, Avg Train Loss: {avg_loss:.4f}")

    model_focal.eval()
    with torch.no_grad():
        logits_test = model_focal(X_test_tensor).squeeze()
        probs_test = torch.sigmoid(logits_test).numpy()

    top_100_indices_focal = np.argsort(-probs_test)[:100]
    top_100_labels_focal = y_test_class[top_100_indices_focal]
    num_class_1_focal = np.sum(top_100_labels_focal > 0.5)
    focal_loss_correct_predictions.append(num_class_1_focal)

# ----- Reporting Results -----
print("\nPerformance across 10 splits:")

print("\nSklearn MLP Model - Correct predictions in top 100:")
for i, correct in enumerate(mlp_correct_predictions, 1):
    print(f"Split {i}: {correct} correct")

print("\nRankNet Model - Correct predictions in top 100:")
for i, correct in enumerate(ranknet_correct_predictions, 1):
    print(f"Split {i}: {correct} correct")

print("\nFocal Loss Model - Correct predictions in top 100:")
for i, correct in enumerate(focal_loss_correct_predictions, 1):
    print(f"Split {i}: {correct} correct")

print(f"\nMLP Average Correct Predictions in Top 100: {np.mean(mlp_correct_predictions)}")
print(f"RankNet Average Correct Predictions in Top 100: {np.mean(ranknet_correct_predictions)}")
print(f"Focal Loss Average Correct Predictions in Top 100: {np.mean(focal_loss_correct_predictions)}")



===== Split 1/10 =====
Focal Loss Model - Epoch 50, Avg Train Loss: 0.0116
Focal Loss Model - Epoch 100, Avg Train Loss: 0.0071

===== Split 2/10 =====
Focal Loss Model - Epoch 50, Avg Train Loss: 0.0108
Focal Loss Model - Epoch 100, Avg Train Loss: 0.0064

===== Split 3/10 =====
Focal Loss Model - Epoch 50, Avg Train Loss: 0.0118
Focal Loss Model - Epoch 100, Avg Train Loss: 0.0077

===== Split 4/10 =====
Focal Loss Model - Epoch 50, Avg Train Loss: 0.0126
Focal Loss Model - Epoch 100, Avg Train Loss: 0.0074

===== Split 5/10 =====
Focal Loss Model - Epoch 50, Avg Train Loss: 0.0100
Focal Loss Model - Epoch 100, Avg Train Loss: 0.0053

===== Split 6/10 =====
Focal Loss Model - Epoch 50, Avg Train Loss: 0.0116
Focal Loss Model - Epoch 100, Avg Train Loss: 0.0070

===== Split 7/10 =====
Focal Loss Model - Epoch 50, Avg Train Loss: 0.0102
Focal Loss Model - Epoch 100, Avg Train Loss: 0.0057

===== Split 8/10 =====
Focal Loss Model - Epoch 50, Avg Train Loss: 0.0115
Focal Loss Model - Ep